In [1]:
import numpy as np

# 初始化棋盘
def create_board():
    return np.zeros((9, 9), dtype=int)

# 检查是否为终止状态
def is_terminal_state(board):
    def check_five(line):
        for i in range(len(line) - 4):
            if abs(sum(line[i:i+5])) == 5:
                return True
        return False

    # 行和列检查
    for i in range(9):
        if check_five(board[i, :]) or check_five(board[:, i]):
            return True

    # 对角线检查
    diags = [board.diagonal(i) for i in range(-8, 9)]
    anti_diags = [np.fliplr(board).diagonal(i) for i in range(-8, 9)]
    for diag in diags + anti_diags:
        if len(diag) >= 5 and check_five(diag):
            return True

    # 检查是否棋盘已满
    if not np.any(board == 0):
        return True

    return False

def evaluate(board):
    """
    改进的评估函数：检测三连模式并增加防守权重。
    """
    score = 0
    patterns = {
        (1, 1, 1, 1, 1): 100000,  # AI五连
        (1, 1, 1, 1, 0): 10000,   # AI四连
        (1, 1, 1, 0, 0): 500,     # AI三连
        (-1, -1, -1, -1, -1): -100000,  # 玩家五连（高危险）
        (-1, -1, -1, -1, 0): -10000,   # 玩家四连（需要阻止）
        (-1, -1, -1, 0, 0): -500,      # 玩家三连（需要优先防守）
    }

    # 只检测棋盘非零区域附近
    non_empty_positions = np.argwhere(board != 0)
    for pos in non_empty_positions:
        row, col = pos
        # 检查行、列、主对角线、副对角线
        lines = [
            board[row, max(0, col-4):min(9, col+5)],  # 行
            board[max(0, row-4):min(9, row+5), col],  # 列
            board.diagonal(col-row),                 # 主对角线
            np.fliplr(board).diagonal(8-row-col)     # 副对角线
        ]
        for line in lines:
            if len(line) >= 5:  # 只有长度大于等于5的线段需要检查模式
                for pattern, value in patterns.items():
                    pattern_str = ''.join(map(str, pattern))
                    line_str = ''.join(map(str, line))
                    score += line_str.count(pattern_str) * value

    return score

# 获取合法动作
def get_legal_moves(board):
    moves = set()
    non_empty_positions = np.argwhere(board != 0)
    for pos in non_empty_positions:
        row, col = pos
        for dr in range(-2, 3):
            for dc in range(-2, 3):
                r, c = row + dr, col + dc
                if 0 <= r < 9 and 0 <= c < 9 and board[r, c] == 0:
                    moves.add((r, c))
    return list(moves)

# Minimax with Alpha-Beta Pruning
def minimax(board, depth, alpha, beta, is_maximizing_player):
    if is_terminal_state(board) or depth == 0:
        return evaluate(board), None

    if is_maximizing_player:
        max_eval = float('-inf')
        best_move = None
        for move in get_legal_moves(board):
            i, j = move
            board[i, j] = 1
            eval, _ = minimax(board, depth - 1, alpha, beta, False)
            board[i, j] = 0
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        best_move = None
        for move in get_legal_moves(board):
            i, j = move
            board[i, j] = -1
            eval, _ = minimax(board, depth - 1, alpha, beta, True)
            board[i, j] = 0
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

# 打印棋盘
def print_board(board):
    for row in board:
        print(' '.join(['X' if cell == 1 else 'O' if cell == -1 else '.' for cell in row]))

# 游戏循环
def play_game():
    board = create_board()
    turn = -1  # 玩家2（O）先手
    while not is_terminal_state(board):
        print_board(board)

        if turn == -1:  # 玩家下棋
            valid_input = False
            while not valid_input:
                try:
                    user_input = input("请输入你的下棋位置（行和列，例如 '1 1' 或 '1,1' 或 '1，1'）：")
                    user_input = user_input.replace('，', ',')
                    if ',' in user_input:
                        row, col = map(int, user_input.split(','))
                    else:
                        row, col = map(int, user_input.split())
                    if board[row, col] == 0:
                        valid_input = True
                    else:
                        print("无效的位置！该位置已被占用。")
                except (ValueError, IndexError):
                    print("无效的输入！请重新输入。")
            board[row, col] = -1
        else:  # AI下棋
            print("AI正在思考...")
            _, ai_move = minimax(board, depth=3, alpha=float('-inf'), beta=float('inf'), is_maximizing_player=True)
            if ai_move:
                board[ai_move[0], ai_move[1]] = 1

        turn = -turn

    print_board(board)
    print("游戏结束！")
    if is_terminal_state(board):
        print("恭喜！您赢了！" if turn == 1 else "很遗憾，AI赢了！")
    else:
        print("平局！")

# 运行游戏
if __name__ == "__main__":
    play_game()


. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
请输入你的下棋位置（行和列，例如 '1 1' 或 '1,1' 或 '1，1'）：4 4
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . O . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
AI正在思考...
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . X . . . .
. . . . O . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
请输入你的下棋位置（行和列，例如 '1 1' 或 '1,1' 或 '1，1'）：4 5
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . X . . . .
. . . . O O . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
AI正在思考...
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . X . . . .
. . . X O O . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
请输入你的下棋位置（行和列，例如 '1 1' 或 '1,1' 或 '1，1'）：4 6
. . . . . . . . .
. . . . . . . . .
. 